In [ ]:
import boto3
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
import tensorflow as tf
from io import BytesIO

In [13]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define local directories (assumed to be within the same instance as the notebook)
base_dir = os.getcwd()

# Local directories for train, validation, and test datasets
local_train_dir = os.path.join(base_dir, 'data/train')
local_valid_dir = os.path.join(base_dir, 'data/valid')
local_test_dir = os.path.join(base_dir, 'data/test')

# Verify that the directories exist (this is optional but helpful for debugging)
print(f"Train directory exists: {os.path.exists(local_train_dir)}")
print(f"Validation directory exists: {os.path.exists(local_valid_dir)}")
print(f"Test directory exists: {os.path.exists(local_test_dir)}")

Train directory exists: True
Validation directory exists: True
Test directory exists: True


In [14]:
import shutil

# Function to remove .ipynb_checkpoints folders
def remove_ipynb_checkpoints(directory):
    for root, dirs, files in os.walk(directory):
        for dir_name in dirs:
            if dir_name == '.ipynb_checkpoints':
                dir_path = os.path.join(root, dir_name)
                shutil.rmtree(dir_path)  # Remove the directory
                print(f"Removed {dir_path}")

# Clean train, valid, and test directories from .ipynb_checkpoints
remove_ipynb_checkpoints(local_train_dir)
remove_ipynb_checkpoints(local_valid_dir)
remove_ipynb_checkpoints(local_test_dir)

Removed /home/ec2-user/SageMaker/data/train/.ipynb_checkpoints
Removed /home/ec2-user/SageMaker/data/valid/.ipynb_checkpoints
Removed /home/ec2-user/SageMaker/data/test/.ipynb_checkpoints


In [15]:

# Define image data generator with preprocessing and augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=30,  # Augmentation: randomly rotate images
    width_shift_range=0.2,  # Augmentation: randomly shift images horizontally
    height_shift_range=0.2,  # Augmentation: randomly shift images vertically
    shear_range=0.2,  # Augmentation: randomly shear images
    zoom_range=0.2,  # Augmentation: randomly zoom into images
    horizontal_flip=True,  # Augmentation: randomly flip images horizontally
    fill_mode='nearest'  # Fill mode for missing pixels after augmentation
)

# No augmentation for validation and test sets, only rescaling
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load all datasets from the subdirectories using flow_from_directory
train_data = train_datagen.flow_from_directory(
    local_train_dir, 
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,
    class_mode='binary'  # Binary classification (Corrosion vs No-corrosion)
)

valid_data = valid_datagen.flow_from_directory(
    local_valid_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    local_test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 270 images belonging to 2 classes.
Found 14 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [52]:
print(train_data.class_indices)


{'Corrosion': 0, 'no-corrosion': 1}


In [16]:
import os

# Check the contents of each directory
print("Train directory classes:", os.listdir(local_train_dir))
print("Valid directory classes:", os.listdir(local_valid_dir))
print("Test directory classes:", os.listdir(local_test_dir))

Train directory classes: ['no-corrosion', 'Corrosion']
Valid directory classes: ['no-corrosion', 'Corrosion']
Test directory classes: ['no-corrosion', 'Corrosion']


### Train Model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [18]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Early stopping to stop training when the validation loss doesn't improve
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)




In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import MobileNetV2

# Load the base pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, validation_data=valid_data, epochs=20, callbacks=[early_stopping])

# Optionally, unfreeze some layers and fine-tune the model


Epoch 1/20
9/9 [==============================] - 21s 1s/step - loss: 1.7766 - accuracy: 0.8815 - val_loss: 0.5598 - val_accuracy: 0.9286
Epoch 2/20
9/9 [==============================] - 5s 545ms/step - loss: 0.2575 - accuracy: 0.9704 - val_loss: 0.6071 - val_accuracy: 0.9286
Epoch 3/20
9/9 [==============================] - 5s 487ms/step - loss: 0.1017 - accuracy: 0.9667 - val_loss: 0.3286 - val_accuracy: 0.9286
Epoch 4/20
9/9 [==============================] - 5s 489ms/step - loss: 0.0085 - accuracy: 0.9963 - val_loss: 0.7216 - val_accuracy: 0.9286
Epoch 5/20
9/9 [==============================] - 5s 492ms/step - loss: 0.0263 - accuracy: 0.9926 - val_loss: 0.6735 - val_accuracy: 0.9286
Epoch 6/20
9/9 [==============================] - 5s 493ms/step - loss: 0.0282 - accuracy: 0.9926 - val_loss: 0.0524 - val_accuracy: 1.0000
Epoch 7/20
9/9 [==============================] - 5s 492ms/step - loss: 0.0434 - accuracy: 0.9889 - val_loss: 0.2463 - val_accuracy: 0.9286
Epoch 8/20
9/9 [======

In [1]:
import io
from PIL import Image
import numpy as np
import tensorflow as tf
import numpy as np
from sagemaker import session, Predictor

# Assuming you have already deployed your model
# Define the endpoint name
endpoint_name = "corrosion-endpoint"


# Create a predictor for the deployed model
predictor = Predictor(endpoint_name)

# Function to load and preprocess an image from a file path
def load_and_preprocess_image(image_path):
    try:
        img = Image.open(image_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img = img.resize((150, 150))
        img_array = np.array(img)
        img_array = img_array.astype(np.float32) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        return img_array
    except Exception as e:
        print(f"Error loading image: {e}")
        return None

2024-09-28 05:17:14.348223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-28 05:17:17.399033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-28 05:17:17.432703: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


KeyboardInterrupt: 

In [ ]:

# Example usage
image_path = "images/pipe.jpg"  # Replace with your actual image path
img = load_and_preprocess_image(image_path=image_path) 
pred_value = round(model.predict(img)[0][0])
prediction = ['Non-Corrosion' if pred_value == 1 else 'Corrosion'][0]

In [43]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                                 
 flatten_2 (Flatten)         (None, 32000)             0         
                                                                 
 dense_4 (Dense)             (None, 64)                2048064   
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4306113 (16.43 MB)
Trainable params: 2048129 (7.81 MB)
Non-trainable params: 2257984 (8.61 MB)
___________

### Save Model

In [30]:
import tarfile 

model.save("export/Servo/1")
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("export")


INFO:tensorflow:Assets written to: export/Servo/1/assets


INFO:tensorflow:Assets written to: export/Servo/1/assets


In [32]:
from sagemaker import session

sm_session = session.Session() 
s3_response = sm_session.upload_data("model.tar.gz", bucket='corrosion-detection-data', key_prefix="model")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### Deploy the Model

In [2]:
import tensorflow as tf

print(tf.__version__)

2024-09-28 05:17:38.420489: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-28 05:17:46.613539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.16.2


In [3]:

import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker import get_execution_role

# the (default) IAM role you created when creating this notebook
role = get_execution_role()

bucket_name = 'corrosion-detection-data'

# Initialize SageMaker session
sagemaker_session = sagemaker.Session()


sagemaker_model = TensorFlowModel(
    model_data=f"s3://{bucket_name}/model/model.tar.gz",
    role=role,
    framework_version="2.16",
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
endpoint_name = 'corrosion-endpoint'
predictor = sagemaker_model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=endpoint_name)

-----!

### Use the Endpoint

In [5]:
import io
from PIL import Image
import numpy as np
import tensorflow as tf
import numpy as np
from sagemaker import session, Predictor

# Assuming you have already deployed your model
# Define the endpoint name
endpoint_name = "corrosion-endpoint"


# Define the endpoint name
endpoint_name = "corrosion-endpoint"

# Create a predictor for the deployed model
predictor = Predictor(endpoint_name)

# Function to load and preprocess an image from a file path
def load_and_preprocess_image(image_path):
    try:
        img = Image.open(image_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img = img.resize((150, 150))
        img_array = np.array(img)
        img_array = img_array.astype(np.float32) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        return img_array
    except Exception as e:
        print(f"Error loading image: {e}")
        return None



In [6]:
predictor

In [7]:
import io
import json
from PIL import Image
import numpy as np
import boto3

# Define the endpoint name
endpoint_name = "corrosion-endpoint"

# Function to load and preprocess an image from a file path
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize((150, 150))  # Resize to the expected input shape
    img_array = np.array(img)
    img_array = img_array.astype(np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Example usage
image_path = "images/pipe.jpg"  # Replace with your actual image path
img = load_and_preprocess_image(image_path=image_path)

# Convert the image to a JSON-compatible format
json_input = {
    "instances": img.tolist()  # Convert to a list for JSON serialization
}

# Use boto3 to send request
client = boto3.client('runtime.sagemaker')
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(json_input),
    ContentType='application/json'  # Use application/json for JSON input
)

# Process the response (this depends on your model's output format)
result = response['Body'].read()
pred_score = json.loads(result)

pred_score = round(pred_score['predictions'][0][0])

prediction = ['Non-corrosion' if pred_score == 1 else 'Corrosion'][0]
prediction

'Corrosion'

### Delete the Endpoint

In [72]:
import boto3

# Create a SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Define the names
endpoint_name = "corrosion-endpoint"
endpoint_config_name = "corrosion-endpoint-config"  # Replace with your endpoint configuration name
model_name = "corrosion-model"  # Replace with your model name

# Step 1: Delete the endpoint
try:
    sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
    print(f"Deleted endpoint: {endpoint_name}")
except Exception as e:
    print(f"Error deleting endpoint: {e}")

# Step 2: Delete the endpoint configuration
try:
    sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
    print(f"Deleted endpoint configuration: {endpoint_config_name}")
except Exception as e:
    print(f"Error deleting endpoint configuration: {e}")

# Step 3: Delete the model
try:
    sagemaker_client.delete_model(ModelName=model_name)
    print(f"Deleted model: {model_name}")
except Exception as e:
    print(f"Error deleting model: {e}")


Deleted endpoint: corrosion-endpoint
Error deleting endpoint configuration: An error occurred (ValidationException) when calling the DeleteEndpointConfig operation: Could not find endpoint configuration "corrosion-endpoint-config".
Error deleting model: An error occurred (ValidationException) when calling the DeleteModel operation: Could not find model "corrosion-model".
